## Topics

- Runnable
- Prompt Template
- Sequential memory
- Output parsing

In [1]:
%pip install langchain langchain-openai

  Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.5.0
    Uninstalling typing-extensions-4.5.0:
      Successfully uninstalled typing-extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.11.0 which is incompatible.
You should consider upgrading via the '/Users/giumast/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain_openai import ChatOpenAI  # pip install langchain-openai

llm = ChatOpenAI(
    openai_api_key="sk-proj-ux2f5nFSFFuOIQvlwJbbT3BlbkFJuu1vCCWysrlGwKay3oGi", 
    temperature=.75, 
    max_tokens=1024, 
    request_timeout=30
)

In [3]:
llm.invoke("test")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3a09b429-4efe-450d-80b8-004264ba0f98-0')

In [ ]:
from langchain.prompts import ChatPromptTemplate  # pip install langchain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Act as a world class Machine Learning engineer. Use english language. End your answers with a reference to the beauty of using data science in any decision you make."),
    ("user", "{input}")
])

# concatenazione del prompt al modello
chain = prompt | llm

## Runnable Interface

To simplify the creation of even very complex event/execution chains, all LangChain components implement a "runnable" protocol through a common interface that allows any component to be used in a standard way. Below are the three main methods:

* **stream** - send partial responses as they are generated
* **invoke** - execute the chain on a single input
* **batch** - execute the chain on multiple inputs

### Input and Output of Main Components
<img src="assets/componenti_io.png" width="600">

One of the advantages of Runnable interfaces is that runnable components can be chained together in execution sequences, allowing the output of one component to automatically become the input to another. The *pipe* command **|** is used for this purpose in LCEL (LangChain Expression Language), enabling the creation of runnable components from other runnable components by configuring them into a sequence that will work synergistically.


In [ ]:
chain.invoke({"input": "hello!"})

# ConversationBufferMemory

[`ConversationBufferMemory`](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html) is a tool in LangChain that helps keep track of a conversation. It stores the messages exchanged between the user and the AI so that the AI can remember what has been said earlier. This helps the AI maintain context and continuity in the conversation.

`ConversationBufferMemory` is a type of sequential memory in Langchain:

<img src="assets/sequential-memory.png" width="300" />


Here’s a basic example of how to add messages to a `ConversationBufferMemory` and how to get back the messages:

In [4]:
from langchain.memory import ConversationBufferMemory

# Create a new conversation memory
memory = ConversationBufferMemory()

# Add user and AI messages to the memory
memory.chat_memory.add_user_message("Hello")
memory.chat_memory.add_ai_message("Hi! How you doin'?")
memory.chat_memory.add_user_message("Fine, thanks.")

print(memory.load_memory_variables({})['history'])

Human: Hello
AI: Hi! How you doin'?
Human: Fine, thanks.


In [5]:
memory = ConversationBufferMemory(return_messages=True)

# Add user and AI messages to the memory
memory.chat_memory.add_user_message("Hello")
memory.chat_memory.add_ai_message("Hi! How you doin'?")
memory.chat_memory.add_user_message("Fine, thanks.")

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello'),
  AIMessage(content="Hi! How you doin'?"),
  HumanMessage(content='Fine, thanks.')]}

# Introduction to PromptTemplate

The `PromptTemplate` is a powerful feature designed to streamline and standardize the creation of prompts for various applications, such as chatbots, automated responses, or data entry forms. It provides a structured format that can be reused across different scenarios, ensuring consistency and efficiency in how inputs are solicited and processed.



In [6]:
# dynamic template and use of a Memory Buffer

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:
{chat}

New question: {question}
Answer:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat")

from langchain.chains import LLMChain

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

/Users/giumast/.pyenv/versions/3.10.0/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
conversation.invoke({"question": "Hello, i lake the orange color."})



> Entering new LLMChain chain...
Prompt after formatting:
Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:


New question: Hello, i lake the orange color.
Answer:

> Finished chain.


{'question': 'Hello, i lake the orange color.',
 'chat': '',
 'text': 'Orange is a beautiful color, just like the beauty of data science lies in its ability to uncover valuable insights and patterns from large amounts of data. Just as the color orange can evoke feelings of warmth and energy, data science can bring to light new perspectives and understanding of complex phenomena. It truly is a fascinating field that continues to amaze and inspire.'}

In [9]:
print(memory.load_memory_variables({})['chat'])

Human: Hello, i lake the orange color.
AI: Orange is a beautiful color, just like the beauty of data science lies in its ability to uncover valuable insights and patterns from large amounts of data. Just as the color orange can evoke feelings of warmth and energy, data science can bring to light new perspectives and understanding of complex phenomena. It truly is a fascinating field that continues to amaze and inspire.


In [10]:
conversation.invoke({"question": "Tell me 3 fruits of my favourite color"})



> Entering new LLMChain chain...
Prompt after formatting:
Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:
Human: Hello, i lake the orange color.
AI: Orange is a beautiful color, just like the beauty of data science lies in its ability to uncover valuable insights and patterns from large amounts of data. Just as the color orange can evoke feelings of warmth and energy, data science can bring to light new perspectives and understanding of complex phenomena. It truly is a fascinating field that continues to amaze and inspire.

New question: Tell me 3 fruits of my favourite color
Answer:

> Finished chain.


{'question': 'Tell me 3 fruits of my favourite color',
 'chat': 'Human: Hello, i lake the orange color.\nAI: Orange is a beautiful color, just like the beauty of data science lies in its ability to uncover valuable insights and patterns from large amounts of data. Just as the color orange can evoke feelings of warmth and energy, data science can bring to light new perspectives and understanding of complex phenomena. It truly is a fascinating field that continues to amaze and inspire.',
 'text': 'Three fruits of your favorite color, orange, are oranges, peaches, and apricots. Just like the vibrant hue of these fruits, data science brings a colorful array of insights and discoveries to light, making it an exciting and dynamic field to explore. The beauty of data science is in its ability to showcase the richness and diversity of information that can be gleaned from analyzing data.'}

In [ ]:
print(memory.load_memory_variables({})['chat'])

## LLM output parsing

<a href="https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/quick_start/" target="_blank">source</a>

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

**Output parsers** are classes that help *structure language model responses*. 

There are two main methods an output parser must implement:

- "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
- "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

- "Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

In [11]:
# OutputParser

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
New question: {question}
Answer:"""

prompt = PromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"question": "Mi piace il colore arancione"})

"Il colore arancione è davvero affascinante, proprio come il mondo della scienza dei dati! Attraverso l'analisi dei dati, possiamo scoprire tendenze nascoste, modelli interessanti e informazioni preziose che possono portare a nuove scoperte e innovazioni. La bellezza della scienza dei dati risiede nella sua capacità di trasformare dati grezzi in conoscenza significativa, aprendo nuove prospettive e possibilità."

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator

class User(BaseModel):
    id: int = Field(description="user identification number")
    name: str = Field(description="user name")
    mail: str = Field(description="user mail address")
    

from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=User)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"id": {"title": "Id", "description": "user identification number", "type": "integer"}, "name": {"title": "Name", "description": "user name", "type": "string"}, "mail": {"title": "Mail", "description": "user mail address", "type": "string"}}, "required": ["id", "name", "mail"]}
```


In [13]:
prompt = PromptTemplate(
    template="Analyze this text\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt

PromptTemplate(input_variables=['query'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "user identification number", "type": "integer"}, "name": {"title": "Name", "description": "user name", "type": "string"}, "mail": {"title": "Mail", "description": "user mail address", "type": "string"}}, "required": ["id", "name", "mail"]}\n```'}, template='Analyze this text\n{format_instructions}\n{query}\n')

In [14]:
chain = prompt | llm | parser

query = "id:1, name: John Doe, e-mail: john.doe@johndoe.com"

chain.invoke({"query": query})

User(id=1, name='John Doe', mail='john.doe@johndoe.com')

In [15]:
chain = prompt | llm | parser

query = "my name is giuseppe mastrandrea, my email is mastrandreagiuseppe@gmail.com and my id is 500"

chain.invoke({"query": query})

User(id=500, name='giuseppe mastrandrea', mail='mastrandreagiuseppe@gmail.com')

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser(pydantic_object=User)

chain = prompt | llm | json_parser

chain.invoke({"query": query})

{'id': 500,
 'name': 'giuseppe mastrandrea',
 'mail': 'mastrandreagiuseppe@gmail.com'}